In [45]:

%use ktor-client
%use serialization
%use dataframe


In [59]:
import io.ktor.client.*
import io.ktor.client.call.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.client.engine.cio.*
import io.ktor.http.HttpHeaders
import kotlinx.coroutines.runBlocking
import kotlinx.serialization.json.*

val client = HttpClient(CIO)

runBlocking {
    val response = client.get("https://relay.notoshi.win") {
        headers { append(HttpHeaders.Accept, "application/nostr+json") }
    }

    val body = response.body<String>()
    //Json.parseToJsonElement(body)
}


```json
{
    "contact": "admin@notoshi.win",
    "description": "nostr-relay: Connecting Thailand Communities #siamstr",
    "name": "relay.notoshi.win",
    "pubkey": "03742c205cb6c8d86031c93bc4a9b3d18484c32c86563fc0e218910a2df9aa5d",
    "software": "git+https://github.com/hoytech/strfry.git",
    "supported_nips": [
        1,
        2,
        4,
        9,
        11,
        12,
        16,
        20,
        22,
        28,
        33,
        40
    ],
    "version": "0.9.6-26-gc0dec7c"
}
```

In [15]:
val boxJson = """
["REQ","hsZEOtaDsENYkP5H-JIWp",{"authors":["e4b2c64f0e4e54abb34d5624cd040e05ecc77f0c467cc46e2cc4d5be98abe3e3"],"#p":["e4b2c64f0e4e54abb34d5624cd040e05ecc77f0c467cc46e2cc4d5be98abe3e3"]}]
"""

val box = boxJson.deserializeJson()
box



```json
[
    "REQ",
    "hsZEOtaDsENYkP5H-JIWp",
    {
        "authors": [
            "e4b2c64f0e4e54abb34d5624cd040e05ecc77f0c467cc46e2cc4d5be98abe3e3"
        ],
        "#p": [
            "e4b2c64f0e4e54abb34d5624cd040e05ecc77f0c467cc46e2cc4d5be98abe3e3"
        ]
    }
]
```

In [64]:
@file:DependsOn("com.squareup.okhttp3:okhttp:4.11.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.15.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.6.0")

import okhttp3.*
import java.util.concurrent.TimeUnit
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import kotlinx.serialization.json.Json
import kotlinx.serialization.json.JsonArray
import kotlinx.serialization.json.jsonArray
import kotlinx.serialization.json.jsonPrimitive
import kotlinx.serialization.json.jsonObject

// Function to convert a JSON string to a JsonArray
fun String.toJsonEltArray(): JsonArray {
    val json = Json { ignoreUnknownKeys = true }
    return json.parseToJsonElement(this).jsonArray
}

// Function to convert any object to a JSON string
fun Any.toJsonString(): String {
    return jacksonObjectMapper().writeValueAsString(this)
}

val client = OkHttpClient.Builder()
    .readTimeout(0, TimeUnit.MILLISECONDS)
    .build()

val sourceList = listOf(
    "wss://relay.notoshi.win",
    "wss://relay.siamstr.com",
    "wss://relay.damus.io",
    "wss://relay.nostr.band",
    "wss://nostr-01.yakihonne.com",
    "wss://nos.lol",
    "wss://purplerelay.com"
)

val publicKey = "e4b2c64f0e4e54abb34d5624cd040e05ecc77f0c467cc46e2cc4d5be98abe3e3"
val reqList = listOf("""["REQ","fffff",{"authors":["$publicKey"],"kinds":[3]}]""")

fun sync() {
    if (sourceList.isNotEmpty()) {
        sourceList.forEach { url ->
            val request: Request = Request.Builder().url(url).build()
            client.newWebSocket(request, SyncData())
        }
    }
}

private inner class SyncData : WebSocketListener() {

    override fun onOpen(webSocket: WebSocket, response: Response) {
        reqList.forEach { request ->
            webSocket.send(request.replace("\$publicKey", publicKey))
        }
    }

    override fun onMessage(webSocket: WebSocket, text: String) {
        val data = text.toJsonEltArray()
        if (data.isNotEmpty() && data[0].jsonPrimitive.content == "EVENT") {
            val res = data[2].jsonObject
            println(res)
        } 
    }

    override fun onClosed(webSocket: WebSocket, code: Int, reason: String) {
        println("WebSocket connection closed: $reason")
    }
}

// Start the synchronization process
sync()
